In Julia, subtypes "inherit" behavior and not implementation details.

What if you really really really want inheritance of fields? There are solutions via metaprogramming. One simple solution is the `@def` macro. 

In [1]:
macro def(name, definition)
  return quote
      macro $(esc(name))()
          esc($(Expr(:quote, definition)))
      end
  end
end

@def (macro with 1 method)

This macro is very simple. What it does is compile-time copy/paste. For example:

In [3]:
@def give_it_a_name begin
  a = 2
  println(a)
end

@give_it_a_name (macro with 1 method)

defines a macro `@give_it_a_name` that will paste in those two lines of code wherever it is used.

One can use this macro to give multiple types the same fields (the same implementation) like so

In [4]:
@def add_generic_fields begin
        method_string::String
        n::Int64
        x::Array{T}
        f_x::T
        f_calls::Int64
        g_calls::Int64
        h_calls::Int64
end

@add_generic_fields (macro with 1 method)

In [ ]:
mutable struct A{T}
    @add_generic_fields
    exlusive_for_A::Float64
    # ... more fields ... 
end

mutable struct B{T}
    @add_generic_fields
    exlusive_for_B::Float64
    another_exlusive_for_B::String
    # ... more fields ... 
end

Because `@def` works at compile-time, there is no cost associated with this. Similar metaprogramming can be used to build an "inheritance feature" for Julia. One package which does this is [ConcreteAbstractions.jl](https://github.com/tbreloff/ConcreteAbstractions.jl) which allows you to add fields to abstract types and make the child types inherit the fields:

```julia
# The abstract type
@base mutable struct AbstractFoo{T}
    a
    b::Int
    c::T
    d::Vector{T}
end

# Inheritance
@extend mutable struct Foo <: AbstractFoo
    e::T
end
```